In [ ]:
# this is the new scripts for the challenge book //Jung Kweon Woo
# some competition notebook codes explanations
# 1. check if a competition has data leakage (in notebooks/discussions)
# 2. top kaggler that submit below 200
# spec: tabular data, standard ML pipline
# RAM 32GB, CPU 8core
# if the data is larger than mem, then use online learning (Vowpal Wabbit)
# spec: image/audio, DL, GPU RAM 8GB (GTX10180)

# Kaggler like using tree-based model, (for tabular data) Gradient Boosting Decision Trees -> XGBoost, LightGBM, CatBoost
# also, bagging-based RandomForest, ExtraTrees -> scilearn
# (for DL models), PyTorch, TensorFlow, Keras
# Sometimes, linear model has similar performance with tree-based model, if cannot loaded in RAM, then use Vowpal Wabbit
# for hyperparameter optimizations, use hyperopt, scikit-optimize, spearmint
# Baseline models means the basic ML pipline: to check if ML pipline correctly work
# To fix the reproductivity, need to fix random_seed
